In [1]:
import os
import numpy as np
import pandas as pd
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
df = pd.read_excel('../input/기후위기_20150701_to_20240630.xlsx')

df_title = df['title']
df_body = df['body']

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10831 entries, 0 to 10830
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   media       10831 non-null  object 
 1   date        10831 non-null  object 
 2   title       10831 non-null  object 
 3   body        10819 non-null  object 
 4   link        10831 non-null  object 
 5   Unnamed: 0  0 non-null      float64
dtypes: float64(1), object(5)
memory usage: 507.8+ KB


In [14]:
sens = df_body
i = 0
# 원본 문서의 첫 번째 원소 확인
sens.iloc[i]

'ㆍ다가올 역사, 서양 문명의 몰락ㆍ나오미 오레스케스 외 지음·홍한별 옮김갈라파고스 | 192쪽 | 1만원아직 다가오지 않은 미래를 상상하는 것은 어쩌면 인간의 본능일지도 모른다. 새로운 기술이 선사할 변화에 대한 상상을 바탕으로 한 사이언스픽션(SF)이 문학·영화·만화를 통틀어 확고한 장르로 자리매김했을 정도이니 말이다.\n\n\n\n300년 후의 미래를 담고 있는 이 책은 가장 ‘리얼’한 SF이다. 지구온난화로 서양 문명이 완전히 몰락한다는 가정을 과학적 근거로 뒷받침하고 있다. 과장된 디스토피아적 상상이라고 치부하기에는 너무 사실적이라 충격을 던진다. 대학에서 과학사를 가르치고 연구하는 저자들은 사실과 학설, 예측과 주장을 적절하게 섞어 쉽고 대중적인 글쓰기를 보여준다.‘300년 후’라니 아득하게만 느껴지는가. 하지만 책에는 놀라운 기시감을 안겨주는 대목이 많다. 기후변화에 관한 정부협의체(IPCC) 등 전문가들의 숱한 경고에도 변함없는 탄소연소 복합체와 셰일가스 붐, 지구온난화 위험이 지나치게 부풀려졌다는 과학계 일각의 주장, 정치 지도자들의 미온적 대응. 바로 지금 벌어지는 일들이다.저자들의 말을 빌리면 우리는 ‘반암흑기’의 한가운데를 지나고 있다. 이후부터는 파국의 시작이다. 2023년이 되면 이례적인 폭염으로 50만명이 사망하지만, 오히려 실내온도를 조절하기 위해 더 많은 에너지를 소비한다. 대기 중 이산화탄소가 2배로 늘어나는 2040년에 이르면 혹서와 가뭄은 일상이 된다. 식량과 물이 바닥나고, 곤충이 폭증하며, 신종 전염병이 퍼진다. 이 무렵 미국과 캐나다가 합병해 ‘북미합중국’을 만든다. 유엔기후변화협약이 실패로 돌아가면서 유엔은 해산하고, 빙하가 수원(水源)인 스위스와 인도가 연대한다.기후변화는 점차 빠른 속도로 생태계와 국경에 균열을 가져온다. 2060년이면 북극의 만년설과 북극곰이 사라진다. 2093년까지는 빙하의 90%가 무너지고, 해수면이 7m 이상 상승한다. 지구 전체 인구의 5명 중 1명꼴인 15억명이 삶의 터전을 빼앗기고 먼 

In [147]:
# sens에서 지정한 패턴이 아닌 글자를 공백으로 변경하고 sens에 재할당
sens = sens.str.replace(pat = '[^가-힣A-Za-z0-9.]|&.+;', repl = '', regex = True)

In [148]:
# sens의 첫 번째 원소와 비교
sens[i]

'다가올역사서양문명의몰락나오미오레스케스외지음홍한별옮김갈라파고스192쪽1만원아직다가오지않은미래를상상하는것은어쩌면인간의본능일지도모른다.새로운기술이선사할변화에대한상상을바탕으로한사이언스픽션SF이문학영화만화를통틀어확고한장르로자리매김했을정도이니말이다.300년후의미래를담고있는이책은가장리얼한SF이다.지구온난화로서양문명이완전히몰락한다는가정을과학적근거로뒷받침하고있다.과장된디스토피아적상상이라고치부하기에는너무사실적이라충격을던진다.대학에서과학사를가르치고연구하는저자들은사실과학설예측과주장을적절하게섞어쉽고대중적인글쓰기를보여준다.300년후라니아득하게만느껴지는가.하지만책에는놀라운기시감을안겨주는대목이많다.기후변화에관한정부협의체IPCC등전문가들의숱한경고에도변함없는탄소연소복합체와셰일가스붐지구온난화위험이지나치게부풀려졌다는과학계일각의주장정치지도자들의미온적대응.바로지금벌어지는일들이다.저자들의말을빌리면우리는반암흑기의한가운데를지나고있다.이후부터는파국의시작이다.2023년이되면이례적인폭염으로50만명이사망하지만오히려실내온도를조절하기위해더많은에너지를소비한다.대기중이산화탄소가2배로늘어나는2040년에이르면혹서와가뭄은일상이된다.식량과물이바닥나고곤충이폭증하며신종전염병이퍼진다.이무렵미국과캐나다가합병해북미합중국을만든다.유엔기후변화협약이실패로돌아가면서유엔은해산하고빙하가수원인스위스와인도가연대한다.기후변화는점차빠른속도로생태계와국경에균열을가져온다.2060년이면북극의만년설과북극곰이사라진다.2093년까지는빙하의90가무너지고해수면이7m이상상승한다.지구전체인구의5명중1명꼴인15억명이삶의터전을빼앗기고먼곳으로이동한다.그렇게서구문명이붕괴한다.책을덮으면막막한심정이든다.그럴땐저자들이묘사하는다급한데도여유를부리는정치지도자들그리고탄소중립사회로옮겨가기는커녕화석연료생산을늘리는부유한국가들을떠올리면어떨까.재앙을멈추거나적어도늦출수있는기회는결국정치에있다는실마리를얻게될것이다.김유진기자yjkimkyunghyang.com모바일경향경향뉴스진경향신문앱모바일웹공식SNS계정경향트위터페이스북세상과경향의소통커뮤니티경향신문www.khan.co.kr무단전재및재배포금지경향신문은한국온

### 형태소 분석

In [149]:
# 내장 불용어 객체 생성
# [참고] 내장 불용어 객체의 stopwords 속성으로 불용어 목록 확인 가능
kiwistop = Stopwords()
words = [('기후', 'NNG'), ('위기', 'NNG'), ('환경', 'NNG'), ('변화', 'NNG'), ('기자', 'NNG'), ('무단', 'NNG'),
         ('재배포', 'NNP'), ('금지', 'NNG'), ('있', 'VA'), ('네이버', 'NNP'), ('전재', 'NNG'), ('및', 'MAG'),
         ('페이스북', 'NNP'), ('트위터', 'NNP'), ('뉴스', 'NNG'), ('구독', 'NNG'), ('메인', 'NNG'), ('서울', 'NNP'),
         ('저작', 'NNG'), ('연합', 'NNG'), ('기사', 'NNG'), ('속보', 'NNG'), ('울산', 'NNP'), ('보기', 'NNG'), 
         ('클릭', 'NNG'), ('채널', 'NNG'), ('배포', 'NNG'), ('경향신문', 'NNP'), ('연합뉴스', 'NNP'), ('뉴시스', 'NNP'),
         ('세계일보', 'NNP'), ('전자신문', 'NNP'), ('문화일보', 'NNP'), ('매경이코노미', 'NNP'), ('KBS', 'NNP'), ('뉴스1', 'NNP'),
         ('프레시안', 'NNP'), ('조선일보', 'NNP'), ('오마이뉴스', 'NNP'), ('MBC', 'NNP'), ('국민일보', 'NNP'), ('미디어오늘', 'NNP'),
         ('한겨레', 'NNP'), ('아시아경제', 'NNP'), ('중앙일보', 'NNP'), ('동아일보', 'NNP'), ('한국일보', 'NNP'), ('노컷뉴스', 'NNP'), 
         ('매일신문', 'NNP'), ('헤럴드경제', 'NNP'), ('부산일보', 'NNP'), ('YTN', 'NNP'), ('파이낸셜뉴스', 'NNP'), ('데일리안', 'NNP'), 
         ('머니S', 'NNP'), ('여성신문', 'NNP'), ('서울신문', 'NNP'), ('중앙SUNDAY', 'NNP'), ('머니투데이', 'NNP'), ('서울경제', 'NNP'), 
         ('디지털타임스', 'NNP'), ('매일경제', 'NNP'), ('이데일리', 'NNP'), ('일다', 'NNP'), ('시사저널', 'NNP'), ('JTBC', 'NNP'), 
         ('주간경향', 'NNP'), ('더팩트', 'NNP'), ('SBS', 'NNP'), ('한겨레21', 'NNP'), ('스포츠서울', 'NNP'), ('한국경제TV', 'NNP'), 
         ('한국경제', 'NNP'), ('지디넷코리아', 'NNP'), ('기자협회보', 'NNP'), ('TV조선', 'NNP'), ('연합뉴스TV', 'NNP'), 
         ('조선비즈', 'NNP'), ('스포츠조선', 'NNP'), ('시사IN', 'NNP'), ('아이뉴스24', 'NNP'), ('강원일보', 'NNP'), 
         ('엑스포츠뉴스', 'NNP'), ('동아사이언스', 'NNP'), ('스타뉴스', 'NNP'), ('한경비즈니스', 'NNP'), ('MBN', 'NNP'), 
         ('마이데일리', 'NNP'), ('SBS Biz', 'NNP'), ('코메디닷컴', 'NNP'), ('스포츠경향', 'NNP'), ('디지털데일리', 'NNP'), 
         ('이코노미스트', 'NNP'),
        ]
kiwistop.add(words)

In [150]:
words = ['경향신문', '연합뉴스', '뉴시스', '세계일보', '전자신문', '문화일보', '매경이코노미', 'KBS', '뉴스1',
              '프레시안', '조선일보', '오마이뉴스', 'MBC', '국민일보', '미디어오늘', '한겨레', '아시아경제', '중앙일보',
              '동아일보', '한국일보', '노컷뉴스', '매일신문', '헤럴드경제', '부산일보', 'YTN', '파이낸셜뉴스',
              '데일리안', '머니S', '여성신문', '서울신문', '중앙SUNDAY', '머니투데이', '서울경제', '디지털타임스',
              '매일경제', '이데일리', '일다', '시사저널', 'JTBC', '주간경향', '더팩트', 'SBS', '한겨레21',
              '스포츠서울', '한국경제TV', '한국경제', '지디넷코리아', '기자협회보', 'TV조선', '연합뉴스TV', '조선비즈',
              '스포츠조선', '시사IN', '아이뉴스24', '강원일보', '엑스포츠뉴스', '동아사이언스', '스타뉴스', '한경비즈니스',
              'MBN', '마이데일리', 'SBS Biz', '코메디닷컴', '스포츠경향', '디지털데일리', '이코노미스트', '재배포']

# 반복문으로 사용자 사전에 단어를 추가
# [참고] score 매개변수에 전달하는 인수의 값이 클수록 하나의 형태소로 처리(기본값: 0)
for word in words:
    kiwi.add_user_word(word = word, tag = 'NNP', score = 15)

In [151]:
# 한글 형태소 분석기 설정
# [참고] typos 매개변수에 'basic'을 지정하면 내장된 오타 정보로 문장 교정
kiwi = Kiwi()

In [152]:
# sens의 첫 번째 원소로 형태소 분석
# [참고] 형태소 분석 결과에서 불용어 제거
kiwi.tokenize(text = sens[i], stopwords = kiwistop)
t = '경향신문연합뉴스뉴시스세계일보전자신문문화일보매경이코노미KBS뉴스1프레시안조선일보오마이뉴스MBC국민일보미디어오늘한겨레아시아경제중앙일보동아일보한국일보노컷뉴스매일신문헤럴드경제부산일보YTN파이낸셜뉴스데일리안머니S여성신문서울신문중앙SUNDAY머니투데이서울경제디지털타임스매일경제이데일리일다시사저널JTBC주간경향더팩트SBS한겨레21스포츠서울한국경제TV한국경제지디넷코리아기자협회보TV조선연합뉴스TV조선비즈스포츠조선시사IN아이뉴스24강원일보엑스포츠뉴스동아사이언스스타뉴스한경비즈니스MBN마이데일리SBSBiz코메디닷컴스포츠경향디지털데일리이코노미스트'
kiwi.tokenize(text=t, stopwords=kiwistop)

[Token(form='KBS', tag='SL', start=29, len=3),
 Token(form='1', tag='SN', start=34, len=1),
 Token(form='MBC', tag='SL', start=48, len=3),
 Token(form='미디어', tag='NNG', start=55, len=3),
 Token(form='오늘', tag='MAG', start=58, len=2),
 Token(form='겨레', tag='NNG', start=61, len=2),
 Token(form='아시아', tag='NNP', start=63, len=3),
 Token(form='경제', tag='NNG', start=66, len=2),
 Token(form='헤럴드', tag='NNG', start=88, len=3),
 Token(form='경제', tag='NNG', start=91, len=2),
 Token(form='YTN', tag='SL', start=97, len=3),
 Token(form='머니', tag='NNG', start=110, len=2),
 Token(form='S', tag='SL', start=112, len=1),
 Token(form='여성', tag='NNG', start=113, len=2),
 Token(form='신문', tag='NNG', start=115, len=2),
 Token(form='중앙', tag='NNG', start=121, len=2),
 Token(form='SUNDAY', tag='SL', start=123, len=6),
 Token(form='일', tag='NR', start=152, len=1),
 Token(form='다', tag='MAG', start=153, len=1),
 Token(form='JTBC', tag='SL', start=158, len=4),
 Token(form='주간', tag='NNG', start=162, len=2),
 To

In [153]:
# sens의 첫 번째 원소로 형태소 분석 결과를 tokens에 할당
# [참고] 사용자 사전에 추가한 단어 반영 여부 확인
tokens = kiwi.tokenize(text = sens[i], stopwords = kiwistop)

# tokens 확인
tokens

[Token(form='다가오', tag='VV', start=0, len=3),
 Token(form='역사', tag='NNG', start=3, len=2),
 Token(form='서양', tag='NNG', start=5, len=2),
 Token(form='문명', tag='NNG', start=7, len=2),
 Token(form='몰락', tag='NNG', start=10, len=2),
 Token(form='나오미', tag='NNP', start=12, len=3),
 Token(form='오레', tag='NNP', start=15, len=2),
 Token(form='스', tag='VV', start=17, len=1),
 Token(form='케스', tag='NNP', start=18, len=2),
 Token(form='외', tag='NNB', start=20, len=1),
 Token(form='짓', tag='VV-I', start=21, len=1),
 Token(form='음', tag='ETN', start=22, len=1),
 Token(form='홍', tag='NNP', start=23, len=1),
 Token(form='한별', tag='NNP', start=24, len=2),
 Token(form='옮기', tag='VV', start=26, len=2),
 Token(form='ᆷ', tag='ETN', start=27, len=1),
 Token(form='갈라파고스', tag='NNP', start=28, len=5),
 Token(form='192', tag='SN', start=33, len=3),
 Token(form='쪽', tag='NNG', start=36, len=1),
 Token(form='1', tag='SN', start=37, len=1),
 Token(form='아직', tag='MAG', start=40, len=2),
 Token(form='다가오', tag=

In [154]:
# tokens의 첫 번째 원소에서 형태소의 형태와 품사 확인
print(f'형태: {tokens[0].form}')
print(f'품사: {tokens[0].tag}')

형태: 다가오
품사: VV


In [155]:
# 텍스트 분석에 사용할 품사(용언과 체언) 목록을 리스트로 생성
pos1, pos2 = ['VV', 'VA'], ['NNG', 'NNP']

In [156]:
# 형태소의 품사가 용언과 체언인 형태소만 선택하고 품사가 용언인 형태소에 
# 종결어미 '다'를 결합하여 리스트로 반환
tokens = [token.form + '다' if token.tag in pos1 else token.form 
          for token in tokens if token.tag in pos1 + pos2]

# tokens 확인
tokens

['다가오다',
 '역사',
 '서양',
 '문명',
 '몰락',
 '나오미',
 '오레',
 '스다',
 '케스',
 '홍',
 '한별',
 '옮기다',
 '갈라파고스',
 '쪽',
 '다가오다',
 '미래',
 '상상',
 '인간',
 '본능',
 '모르다',
 '기술',
 '선사',
 '상상',
 '바탕',
 '사이언스',
 '픽션',
 '문학',
 '영화',
 '만화',
 '통틀다',
 '장르',
 '자리매김',
 '정도',
 '후',
 '미래',
 '담다',
 '책',
 '리얼',
 '지구',
 '온난',
 '서양',
 '문명',
 '몰락',
 '가정',
 '과학',
 '근거',
 '뒷받침',
 '과장',
 '디스토피아',
 '상상',
 '치부',
 '사실',
 '충격',
 '던지다',
 '대학',
 '과학사',
 '가르치다',
 '연구',
 '저자',
 '사실',
 '학설',
 '예측',
 '주장',
 '섞다',
 '대중',
 '글',
 '쓰다',
 '보이다',
 '후',
 '느끼다',
 '책',
 '기시감',
 '안기다',
 '대목',
 '많다',
 '관하다',
 '정부',
 '협의체',
 '전문가',
 '숱하다',
 '경고',
 '변하다',
 '탄소',
 '연소',
 '복합체',
 '셰일가스',
 '붐',
 '지구',
 '온난',
 '위험',
 '지나치다',
 '부풀리다',
 '과학',
 '일각',
 '주장',
 '정치',
 '지도자',
 '미온',
 '대응',
 '벌어지다',
 '빌리다',
 '반',
 '암흑기',
 '가운데',
 '지나다',
 '후',
 '파국',
 '시작',
 '이례',
 '폭염',
 '사망',
 '실내',
 '온도',
 '조절',
 '많다',
 '에너지',
 '소비',
 '대기',
 '이산화탄소',
 '배',
 '늘어나다',
 '혹서',
 '가뭄',
 '일상',
 '식량',
 '물',
 '바닥나다',
 '곤충',
 '폭증',
 '신종',
 '전염병',
 '퍼지다',
 '미국',
 '캐나다',
 '합병',
 '북미',
 '합

In [157]:
# 말뭉치를 저장할 빈 리스트 생성
corpus = list()

# sens의 원소(문서)를 형태소 분석하고 일부 품사를 선택하여 말뭉치에 추가
for sen in sens:
    tokens = kiwi.tokenize(text = sen, stopwords = kiwistop)
    tokens = [token.form + '다' if token.tag in pos1 else token.form 
              for token in tokens if token.tag in pos1 + pos2]
    corpus.append(tokens)

In [158]:
# 형태소 분석 결과(corpus)의 처음 5개 원소 확인
# [참고] 원본 문서와 결과를 비교하여 사용자 사전에 추가하는 것을 반복해야 함
for i in range(5):
    print(corpus[i])

['다가오다', '역사', '서양', '문명', '몰락', '나오미', '오레', '스다', '케스', '홍', '한별', '옮기다', '갈라파고스', '쪽', '다가오다', '미래', '상상', '인간', '본능', '모르다', '기술', '선사', '상상', '바탕', '사이언스', '픽션', '문학', '영화', '만화', '통틀다', '장르', '자리매김', '정도', '후', '미래', '담다', '책', '리얼', '지구', '온난', '서양', '문명', '몰락', '가정', '과학', '근거', '뒷받침', '과장', '디스토피아', '상상', '치부', '사실', '충격', '던지다', '대학', '과학사', '가르치다', '연구', '저자', '사실', '학설', '예측', '주장', '섞다', '대중', '글', '쓰다', '보이다', '후', '느끼다', '책', '기시감', '안기다', '대목', '많다', '관하다', '정부', '협의체', '전문가', '숱하다', '경고', '변하다', '탄소', '연소', '복합체', '셰일가스', '붐', '지구', '온난', '위험', '지나치다', '부풀리다', '과학', '일각', '주장', '정치', '지도자', '미온', '대응', '벌어지다', '빌리다', '반', '암흑기', '가운데', '지나다', '후', '파국', '시작', '이례', '폭염', '사망', '실내', '온도', '조절', '많다', '에너지', '소비', '대기', '이산화탄소', '배', '늘어나다', '혹서', '가뭄', '일상', '식량', '물', '바닥나다', '곤충', '폭증', '신종', '전염병', '퍼지다', '미국', '캐나다', '합병', '북미', '합중국', '만드다', '유엔', '협약', '실패', '돌아가다', '유엔', '해산', '빙하', '수원', '스위스', '인도', '연대', '.기후', '빠르다', '속도', '생태', '국경', '균열', '가져오다', '북극', '만년

In [159]:
# 반복문으로 원본 문서의 처음 5행을 출력
for i in range(5):
    print(df['body'].iloc[i], '\n')

ㆍ다가올 역사, 서양 문명의 몰락ㆍ나오미 오레스케스 외 지음·홍한별 옮김갈라파고스 | 192쪽 | 1만원아직 다가오지 않은 미래를 상상하는 것은 어쩌면 인간의 본능일지도 모른다. 새로운 기술이 선사할 변화에 대한 상상을 바탕으로 한 사이언스픽션(SF)이 문학·영화·만화를 통틀어 확고한 장르로 자리매김했을 정도이니 말이다.



300년 후의 미래를 담고 있는 이 책은 가장 ‘리얼’한 SF이다. 지구온난화로 서양 문명이 완전히 몰락한다는 가정을 과학적 근거로 뒷받침하고 있다. 과장된 디스토피아적 상상이라고 치부하기에는 너무 사실적이라 충격을 던진다. 대학에서 과학사를 가르치고 연구하는 저자들은 사실과 학설, 예측과 주장을 적절하게 섞어 쉽고 대중적인 글쓰기를 보여준다.‘300년 후’라니 아득하게만 느껴지는가. 하지만 책에는 놀라운 기시감을 안겨주는 대목이 많다. 기후변화에 관한 정부협의체(IPCC) 등 전문가들의 숱한 경고에도 변함없는 탄소연소 복합체와 셰일가스 붐, 지구온난화 위험이 지나치게 부풀려졌다는 과학계 일각의 주장, 정치 지도자들의 미온적 대응. 바로 지금 벌어지는 일들이다.저자들의 말을 빌리면 우리는 ‘반암흑기’의 한가운데를 지나고 있다. 이후부터는 파국의 시작이다. 2023년이 되면 이례적인 폭염으로 50만명이 사망하지만, 오히려 실내온도를 조절하기 위해 더 많은 에너지를 소비한다. 대기 중 이산화탄소가 2배로 늘어나는 2040년에 이르면 혹서와 가뭄은 일상이 된다. 식량과 물이 바닥나고, 곤충이 폭증하며, 신종 전염병이 퍼진다. 이 무렵 미국과 캐나다가 합병해 ‘북미합중국’을 만든다. 유엔기후변화협약이 실패로 돌아가면서 유엔은 해산하고, 빙하가 수원(水源)인 스위스와 인도가 연대한다.기후변화는 점차 빠른 속도로 생태계와 국경에 균열을 가져온다. 2060년이면 북극의 만년설과 북극곰이 사라진다. 2093년까지는 빙하의 90%가 무너지고, 해수면이 7m 이상 상승한다. 지구 전체 인구의 5명 중 1명꼴인 15억명이 삶의 터전을 빼앗기고 먼 곳으로 이

In [160]:
# corpus의 각 원소(리스트)를 하나의 문자열로 결합하여 corpus에 재할당
# [참고] 문서-단어 행렬 생성하려면 corpus 원소가 문자열이어야 함
corpus = [' '.join(i) for i in corpus]

# corpus의 처음 5개 확인
corpus[0:1]

['다가오다 역사 서양 문명 몰락 나오미 오레 스다 케스 홍 한별 옮기다 갈라파고스 쪽 다가오다 미래 상상 인간 본능 모르다 기술 선사 상상 바탕 사이언스 픽션 문학 영화 만화 통틀다 장르 자리매김 정도 후 미래 담다 책 리얼 지구 온난 서양 문명 몰락 가정 과학 근거 뒷받침 과장 디스토피아 상상 치부 사실 충격 던지다 대학 과학사 가르치다 연구 저자 사실 학설 예측 주장 섞다 대중 글 쓰다 보이다 후 느끼다 책 기시감 안기다 대목 많다 관하다 정부 협의체 전문가 숱하다 경고 변하다 탄소 연소 복합체 셰일가스 붐 지구 온난 위험 지나치다 부풀리다 과학 일각 주장 정치 지도자 미온 대응 벌어지다 빌리다 반 암흑기 가운데 지나다 후 파국 시작 이례 폭염 사망 실내 온도 조절 많다 에너지 소비 대기 이산화탄소 배 늘어나다 혹서 가뭄 일상 식량 물 바닥나다 곤충 폭증 신종 전염병 퍼지다 미국 캐나다 합병 북미 합중국 만드다 유엔 협약 실패 돌아가다 유엔 해산 빙하 수원 스위스 인도 연대 .기후 빠르다 속도 생태 국경 균열 가져오다 북극 만년설 북극곰 사라지다 빙하 무너지다 해수면 이상 상승 지구 전체 인구 삶 터전 빼앗기다 곳 이동 서구 문명 붕괴 책 덮다 심정 들다 저자 묘사 여유 부리다 정치 지도자 탄소 중립 사회 옮기다 화석 연료 생산 늘리다 부유 국가 떠올리다 재앙 멈추다 늦추다 기회 정치 실마리 김유진 모바일 경향 경향 지다 앱 모바일 웹 공식 계정 경향 세상 경향 소통 커뮤니티 한국온라인신문협회 디지털 이용 규칙 행사']

In [161]:
# 문서별 단어의 tf를 계산하는 객체 생성
# [참고] TfidfVectorizer()는 단어의 tf-idf를 반환
cv = CountVectorizer()

In [162]:
# 단어의 tf를 성분으로 갖는 문서-단어 행렬 생성
dtm = cv.fit_transform(raw_documents = corpus).toarray()

# dtm 확인
# [참고] dtm은 2차원 행렬임
dtm

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [163]:
# cv 객체에 저장된 단어 목록 확인
# [참고] 알파벳이 모두 소문자로 바뀌었음
cv.get_feature_names_out()

array(['0를', '0으', '0으로', ..., '힘주다', '힘차다', '힙합'], dtype=object)

In [164]:
# dtm을 데이터프레임으로 변환
dtm = pd.DataFrame(data = dtm, columns = cv.get_feature_names_out())

In [165]:
# dtm의 열이름에서 알파벳을 대문자로 변경
dtm.columns = dtm.columns.str.upper()

In [166]:
# dtm의 처음 5행 확인
dtm.head()

,0를,0으,0으로,0이,0제로,16스웨덴,19를,19와,19으,19펜데,...,힐링,힐스,힘내다,힘들다,힘세다,힘쓰다,힘없다,힘주다,힘차다,힙합
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [167]:
# dtm의 행 개수와 열 개수 확인
dtm.shape

(2590, 21618)

In [168]:
# dtm의 단어별 tf 합계를 내림차순 정렬하고 tfs에 할당
tfs = dtm.sum().sort_values(ascending = False)

In [169]:
# tfs의 상위 10개 및 하위 10개 확인
display(tfs.head(n = 100))
display(tfs.tail(n = 100))

탄소     5280
대응     4699
에너지    4002
행동     3331
정부     3202
       ... 
분야      843
나오다     843
기술      842
위원회     841
재배      831
Length: 100, dtype: int64

승패       1
승하       1
승하다      1
승하차      1
승학초      1
        ..
숟가락      1
술렁이다     1
숨결       1
숨넘어가다    1
스테이      1
Length: 100, dtype: int64

In [170]:
# tfs가 2 이하인 인덱스(단어)를 dtm에서 제거
# [참고] 단어 빈도수를 높일수록 dtm의 열(차원) 개수가 감소함
threshold = 20
dtm = dtm.drop(columns = tfs.loc[tfs.le(threshold)].index)

In [171]:
# dtm의 행 개수와 열 개수 확인
# [참고] 단어 빈도수가 매우 작은 일부 단어를 삭제함으로써 차원(열) 축소 가능
# [참고] dtm의 차원을 축소하면 행렬곱 연산을 빠르게 실행할 수 있음
dtm.shape

(2590, 3333)

In [172]:
# corpus, tfs 및 dtm을 하나의 pkl 파일로 저장
pd.to_pickle(obj = [corpus, tfs, dtm], filepath_or_buffer = '2015to2021_Text_Prep.pkl')